In [2]:
text="""
경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n
그런 식으로 하면 말이 안된다\n
새로 태어난 말이 배가 고프다\n
"""


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 단어 집합을 만들고 단어에 고유한 숫자 인덱스를 부여(1부터 시작)
t = Tokenizer()
t.fit_on_texts([text])
# 원핫인코딩에서는 0부터 시작하므로 미리 1을 더함
vocab_size = len(t.word_index) + 1
print(' 단어 집합의 크기 : %d' % vocab_size) 

 단어 집합의 크기 : 20


In [4]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11, '그런': 12, '식으로': 13, '하면': 14, '안된다': 15, '새로': 16, '태어난': 17, '배가': 18, '고프다': 19}


In [5]:
sequences = list()
for line in text.split('\n'): # 문장별로 나누기
    # 단어의 인덱스로만 구성된 새로운 리스트
    encoded = t.texts_to_sequences([line])[0] # 문장을 숫자로 변환
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
print(' 학습에 사용할 샘플의 개수 : %d' % len(sequences))

 학습에 사용할 샘플의 개수 : 19


In [7]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11], [12, 13], [12, 13, 14], [12, 13, 14, 1], [12, 13, 14, 1, 15], [16, 17], [16, 17, 1], [16, 17, 1, 18], [16, 17, 1, 18, 19]]


In [8]:
max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print(' 샘플의 최대 길이 : {}'.format(max_len))

 샘플의 최대 길이 : 6


In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
a=[[1,2,3],[4,5],[6,7,8,9]]
print(a)
# 같은 길이로 맞추고 길이가 부족한 경우 왼쪽에 공백을 채움
b=pad_sequences(a)
print(b)
# 오른쪽에 공백을 채움
c=pad_sequences(a,padding='post')
print(c)
# 오른쪽에 1을 채움
d=pad_sequences(a,padding='post',value=1)
print(d)
# 최대 길이를 3으로 설정 (왼쪽이 잘림, truncating='pre') 
e=pad_sequences(a,padding='post',maxlen=3)
print(e)
# 최대 길이를 3으로 설정 (오른쪽이 잘림)
f=pad_sequences(a,padding='post',maxlen=3,truncating='post')
print(f)


[[1, 2, 3], [4, 5], [6, 7, 8, 9]]
[[0 1 2 3]
 [0 0 4 5]
 [6 7 8 9]]
[[1 2 3 0]
 [4 5 0 0]
 [6 7 8 9]]
[[1 2 3 1]
 [4 5 1 1]
 [6 7 8 9]]
[[1 2 3]
 [4 5 0]
 [7 8 9]]
[[1 2 3]
 [4 5 0]
 [6 7 8]]


In [10]:
# 최대 사이즈에 맞추어 빈칸에 0으로 채움                우측정렬
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [11]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]
 [ 0  0  0  0 12 13]
 [ 0  0  0 12 13 14]
 [ 0  0 12 13 14  1]
 [ 0 12 13 14  1 15]
 [ 0  0  0  0 16 17]
 [ 0  0  0 16 17  1]
 [ 0  0 16 17  1 18]
 [ 0 16 17  1 18 19]]


In [12]:
import numpy as np
sequences = np.array(sequences)
# 리스트의 마지막 값을 제외하고 저장
X = sequences[:,:-1]
# 리스트의 마지막 값만 저장
y = sequences[:,-1]

In [13]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]
 [ 0  0  0  0 12]
 [ 0  0  0 12 13]
 [ 0  0 12 13 14]
 [ 0 12 13 14  1]
 [ 0  0  0  0 16]
 [ 0  0  0 16 17]
 [ 0  0 16 17  1]
 [ 0 16 17  1 18]]


In [14]:
print(y) # 모든 샘플에 대한 레이블 출력


[ 3  1  4  5  1  7  1  9 10  1 11 13 14  1 15 17  1 18 19]


In [15]:
from keras.utils import np_utils
#원핫인코딩
y = np_utils.to_categorical(y, num_classes=vocab_size)

In [16]:
print(y)


[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.

In [17]:
max_len

6

In [19]:
from tensorflow.keras.models import Sequential
from keras.layers import Embedding
# Embedding
model = Sequential()
# Embedding(단어의개수, 벡터크기, 입력사이즈)
# 각 단어에 대한 정수 인코딩
words={'I':0, 'am':1, 'a':2, 'boy':3}
# Embedding(단어개수, 벡터크기, 입력사이즈(최대단어개수))
# keras 신경망에 word2vec벡터를 입력하는 방법
model.add(Embedding(5, 1, input_length=1))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 가중치값
embeddings = model.layers[0].get_weights()[0]
words_embeddings = {w:embeddings[idx] for w, idx in words.items()}
print(words_embeddings)
print(words_embeddings['boy'])

{'I': array([0.00465784], dtype=float32), 'am': array([0.03207755], dtype=float32), 'a': array([-0.00689923], dtype=float32), 'boy': array([0.03530315], dtype=float32)}
[0.03530315]


In [20]:
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN
model = Sequential()
# y를 분리하였으므로 X의 길이는 5
# Embedding(단어개수, 벡터크기, 입력사이즈(최대단어개수))
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(SimpleRNN(32)) # 출력노드 32개
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 10)             200       
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1376      
                                                                 
 dense (Dense)               (None, 20)                660       
                                                                 
Total params: 2,236
Trainable params: 2,236
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 2s - loss: 2.9948 - accuracy: 0.0526 - 2s/epoch - 2s/step
Epoch 2/200
1/1 - 0s - loss: 2.9850 - accuracy: 0.0526 - 20ms/epoch - 20ms/step
Epoch 3/200
1/1 - 0s - loss: 2.9752 - accuracy: 0.2105 - 24ms/epoch - 24ms/step
Epoch 4/200
1/1 - 0s - loss: 2.9655 - accuracy: 0.2105 - 22ms/epoch - 22ms/step
Epoch 5/200
1/1 - 0s - loss: 2.9556 - accuracy: 0.2632 - 19ms/epoch - 19ms/step
Epoch 6/200
1/1 - 0s - loss: 2.9455 - accuracy: 0.3158 - 20ms/epoch - 20ms/step
Epoch 7/200
1/1 - 0s - loss: 2.9351 - accuracy: 0.3158 - 17ms/epoch - 17ms/step
Epoch 8/200
1/1 - 0s - loss: 2.9244 - accuracy: 0.3158 - 20ms/epoch - 20ms/step
Epoch 9/200
1/1 - 0s - loss: 2.9131 - accuracy: 0.3158 - 18ms/epoch - 18ms/step
Epoch 10/200
1/1 - 0s - loss: 2.9014 - accuracy: 0.3158 - 18ms/epoch - 18ms/step
Epoch 11/200
1/1 - 0s - loss: 2.8890 - accuracy: 0.3158 - 16ms/epoch - 16ms/step
Epoch 12/200
1/1 - 0s - loss: 2.8760 - accuracy: 0.3158 - 19ms/epoch - 19ms/step
Epoch 13/200
1/1 - 0s - loss: 2.8622 - ac

In [1]:
# 모델, 토크나이저, 현재 단어, 반복할 횟수
def sentence_generation(model, t, current_word, n):
    # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    init_word = current_word
    sentence = ''
    for _ in range(n): # n번 반복
        # 현재 단어에 대한 정수 인코딩
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre') # 데이터에 대한 패딩
        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        pred=model.predict(encoded)
        result=np.argmax(pred,axis=1)
        for word, index in t.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result: 
                break # 해당 단어가 예측 단어이므로 break
        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' ' + word # 단어 누적
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    sentence = init_word + sentence
    return sentence


In [2]:
print(sentence_generation(model, t, '경마장에', 4)) 
# '경마장에'라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측

NameError: name 'model' is not defined

In [26]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

1/1 [==============================] - 0s 34ms/step
그의 말이 법이다


In [27]:
print(sentence_generation(model, t, '가는', 5)) # 5번 예측


1/1 [==============================] - 0s 20ms/step
가는 말이 고와야 오는 말이 곱다


In [29]:
print(sentence_generation(model, t, '그런', 4)) # 4번 예측


1/1 [==============================] - 0s 22ms/step
그런 식으로 하면 말이 안된다


In [28]:
print(sentence_generation(model, t, '새로', 4)) # 4번 예측

1/1 [==============================] - 0s 28ms/step
새로 태어난 말이 배가 고프다
